In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20379744
paper_name = 'mattiazzi_petrovic_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/ATX_SDL_combined_MattiazziUsaj_MGG2010.xlsx', 
                              sheet_name='ATX_SDL_combined_MattiazziUsaj_')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4807 x 8


In [7]:
original_data.head()

,Plate,Row,Column,Name,ORF,SDL1_R(Atx),SDL2_R(Atx),Average_R(Atx)
0,1,1,2,HIS3,YOR202W,NaN,0.913856,0.913856
1,1,1,3,HIS3,YOR202W,NaN,0.973750,0.973750
2,1,1,4,HIS3,YOR202W,NaN,0.942905,0.942905
3,1,1,5,HIS3,YOR202W,NaN,1.152531,1.152531
4,1,1,6,HIS3,YOR202W,NaN,0.821557,0.821557


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Plate, Row, Column, Name, ORF, SDL1_R(Atx), SDL2_R(Atx), Average_R(Atx), orf]
Index: []


In [12]:
original_data['data'] = original_data['Average_R(Atx)'].astype(float)

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(4103, 1)

# Load data (2)

In [17]:
original_data2 = pd.read_excel('raw_data/438_2010_533_MOESM2_ESM.xlsx', 
                              sheet_name='SDL screen 1')

In [18]:
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 154 x 6


In [19]:
original_data2.head()

,ORF,Name,Unnamed: 2,Unnamed: 3,Unnamed: 4,lethal
0,YMR154C,RIM13,-3.0,NaN,NaN,very sick
1,YOL108C,INO4,-3.0,NaN,NaN,sick
2,YIR023W,DAL81,-3.0,NaN,NaN,NaN
3,YDL074C,BRE1,-3.0,NaN,NaN,NaN
4,YHR121W,YHR121W,-3.0,NaN,NaN,NaN


In [20]:
original_data2['orf'] = original_data2['ORF'].astype(str)

In [21]:
# Eliminate all white spaces & capitalize
original_data2['orf'] = clean_orf(original_data2['orf'])

In [22]:
# Translate to ORFs 
original_data2['orf'] = translate_sc(original_data2['orf'], to='orf')

In [23]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orf'])
print(original_data2.loc[~t,])

             ORF Name  Unnamed: 2  Unnamed: 3  Unnamed: 4 lethal  orf
index_input                                                          
152          NaN  NaN         NaN         NaN         NaN    NaN  NAN
153          152  NaN         NaN         NaN         NaN    NaN  152


In [24]:
original_data2 = original_data2.loc[t,:]

In [25]:
original_data2['data'] = original_data2['Unnamed: 2'].astype(float)

In [26]:
original_data2.set_index('orf', inplace=True)

In [27]:
original_data2 = original_data2[['data']].copy()

In [28]:
original_data2 = original_data2.groupby(original_data2.index).mean()

In [29]:
original_data2.shape

(152, 1)

# Merge

In [30]:
original_data = original_data.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [31]:
original_data.shape

(4125, 2)

In [32]:
original_data.head()

,data_1,data_2
orf,,
YAL002W,0.287220,NaN
YAL004W,1.217207,NaN
YAL005C,1.454849,NaN
YAL007C,0.970050,NaN
YAL008W,1.460477,NaN


In [33]:
# Use data_1 as a tested space for data_2

In [38]:
original_data.isnull().sum(axis=0)

data_1      22
data_2    3973
dtype: int64

In [40]:
original_data.loc[original_data['data_2'].isnull(),'data_2'] = 0

In [41]:
original_data.loc[original_data['data_1'].isnull(),]

,data_1,data_2
orf,,
YAL035W,NaN,-3.0
YCL008C,NaN,-3.0
YCR004C,NaN,-3.0
YDR200C,NaN,-3.0
YEL024W,NaN,-3.0
YEL046C,NaN,-3.0
YER040W,NaN,-3.0
YGR057C,NaN,-2.0
YJL006C,NaN,-1.0


# Prepare the final dataset

In [42]:
data = original_data.copy()

In [43]:
dataset_ids = [16127,16126]
datasets = datasets.reindex(index=dataset_ids)

In [44]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [45]:
data.head()

dataset_id,16127,16126
data_type,value,value
orf,,
YAL002W,0.287220,0.0
YAL004W,1.217207,0.0
YAL005C,1.454849,0.0
YAL007C,0.970050,0.0
YAL008W,1.460477,0.0


## Subset to the genes currently in SGD

In [46]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [47]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16127,16126
,data_type,value,value
gene_id,orf,,
2,YAL002W,0.287220,0.0
1863,YAL004W,1.217207,0.0
4,YAL005C,1.454849,0.0
5,YAL007C,0.970050,0.0
6,YAL008W,1.460477,0.0


# Normalize

In [48]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [49]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [50]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16127,16126,16127,16126
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,0.287220,0.0,-2.374026,0.0
1863,YAL004W,1.217207,0.0,1.066954,0.0
4,YAL005C,1.454849,0.0,1.946237,0.0
5,YAL007C,0.970050,0.0,0.152467,0.0
6,YAL008W,1.460477,0.0,1.967059,0.0


# Print out

In [51]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [60]:
# from IO.save_data_to_db3 import *

In [61]:
# save_data_to_db(data_all, paper_pmid)